In [33]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator
from scipy import stats

In [34]:
# loading data
raw = pd.read_csv("forestfires.csv")
data = pd.DataFrame(raw)
data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [47]:
# prepocessing data
data_binned = data.copy()
data_binned["FFMC"] = pd.qcut(data["FFMC"], 4)#[0,20,40,60,80,100])
data_binned["DMC"] = pd.qcut(data["DMC"], 4)#[0,100,200,300])
data_binned["ISI"] = pd.qcut(data["ISI"], 4)#[0,20,40,60])
data_binned["temp"] = pd.qcut(data["temp"], 4)#[0,20,40])
data_binned["RH"] = pd.qcut(data["RH"], 4)#[0,20,40,60,80,100])
data_binned["wind"] = pd.cut(data["wind"], 5)
data_binned["rain"] = pd.cut(data["rain"], 2)
data_binned["area"] = pd.qcut(data["area"], 2)#[-2,0,1100])
data_binned = data_binned.drop('DC', 1)
data_binned.head()

,X,Y,month,day,FFMC,DMC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,"(18.699, 90.2]","(1.099, 68.6]","(-0.001, 6.5]","(2.199, 15.5]","(42.0, 53.0]","(5.8, 7.6]","(-0.0064, 3.2]","(-0.001, 0.52]"
1,7,4,oct,tue,"(90.2, 91.6]","(1.099, 68.6]","(6.5, 8.4]","(15.5, 19.3]","(14.999, 33.0]","(0.391, 2.2]","(-0.0064, 3.2]","(-0.001, 0.52]"
2,7,4,oct,sat,"(90.2, 91.6]","(1.099, 68.6]","(6.5, 8.4]","(2.199, 15.5]","(14.999, 33.0]","(0.391, 2.2]","(-0.0064, 3.2]","(-0.001, 0.52]"
3,8,6,mar,fri,"(91.6, 92.9]","(1.099, 68.6]","(8.4, 10.8]","(2.199, 15.5]","(53.0, 100.0]","(2.2, 4.0]","(-0.0064, 3.2]","(-0.001, 0.52]"
4,8,6,mar,sun,"(18.699, 90.2]","(1.099, 68.6]","(8.4, 10.8]","(2.199, 15.5]","(53.0, 100.0]","(0.391, 2.2]","(-0.0064, 3.2]","(-0.001, 0.52]"


In [48]:
# split into train and test set
data_binned = data_binned.sample(frac=1) #shuffle
split = int(len(data_binned)/5)
data_binned_train = data_binned[:split] #4/5
data_binned_test = data_binned[split:] #1/5
test_labels = data_binned_test["area"]
data_binned_test = data_binned_test.drop("area", axis=1)

In [49]:
# create graph for the model
G = BayesianModel()
G.add_nodes_from(data_binned)
G.add_edges_from([("month","temp"),
                  ("month","wind"),
                  ("month","RH"),
                  ("month", "rain"),
                  #("month", "area"), 
                  ("wind","ISI"),
                  ("wind","FFMC"),
                  #("wind","area"),
                  ("temp","FFMC"),
                  ("temp","DMC"),
                  #("temp","area"),
                  ("RH","FFMC"),
                  ("RH","DMC"),
                  #("RH","area"),
                  ("rain","DMC"),
                  ("rain","FFMC"),
                  #("rain","area"),
                  ("FFMC","ISI"),
                  ("FFMC","area"),
                  ("DMC","area"),
                  ("ISI","area"),
                  ("X","area"),
                  ("Y","area"),
                  ("day","area")
                 ])

In [50]:
(bins, freq) = ([],[])
(variables, bf) = ([],(bins, freq))
for col in data_binned:
    #print(col)
    vals, counts = np.unique(data_binned.get(col), return_counts=True)
    perc = counts/np.sum(counts)*100
    threshold = np.sum(counts)*0.1 
    results = dict(zip(vals, counts)) #replace perc with counts for count
    #print(pd.Series(results))
    variables.append(col)
    bins.append(vals)
    freq.append(counts)
    #print('sum = ', np.sum(perc))
    #print("perc" , perc)

def freq_bins(bins, threshold):
    """get most frequent bins (>10% of dataset)"""
    #This stores the bins and frequencies of the following variables: wind, month, rain, RH, temp
    (bins_wind, freq_wind) = ([],[])
    (bins_month,freq_month) = ([],[])
    (bins_rain,freq_rain) = ([],[])
    (bins_RH,freq_RH) = ([],[])
    (bins_temp,freq_temp) = ([],[])
    
    #print bins  
    #print len(bins)
    #print len(bins[0])
    for i in range(len(bins)):
        #print(variables[i])
        #for j in range(len(bins[i])):
        if(variables[i]=="wind"):
            ind = [j for (j, val) in enumerate(freq[i]) if val > threshold]
            (bins_wind,freq_wind) = ([ bins[i][j] for j in ind], [x for x in freq[i] if x > threshold])
        if(variables[i]=="rain"): 
            ind = [j for (j, val) in enumerate(freq[i]) if val > threshold]
            (bins_rain,freq_rain) = ([bins[i][j] for j in ind], [x for x in freq[i] if x > threshold])
        if(variables[i]=="RH"): 
            ind = [j for (j, val) in enumerate(freq[i]) if val > threshold]
            (bins_RH,freq_RH) = ([bins[i][j] for j in ind], [x for x in freq[i] if x > threshold])
        if(variables[i]=="temp"): 
            ind = [j for (j, val) in enumerate(freq[i]) if val > threshold]
            (bins_temp,freq_temp) = ([bins[i][j] for j in ind], [x for x in freq[i] if x > threshold])
        if(variables[i]=="month"): 
            ind = [j for (j, val) in enumerate(freq[i]) if val > threshold]
            (bins_month,freq_month) = ([bins[i][j] for j in ind], [x for x in freq[i] if x > threshold])
        if(variables[i]=="area"):
            (bins_area,freq_area) = (bins[i],freq[i])
        #print(bins[i],freq[i])
        
        chi_data = [(bins_wind, freq_wind),(bins_month,freq_month),(bins_rain,freq_rain),(bins_RH,freq_RH),(bins_temp,freq_temp),(bins_area,freq_area)]
        return chi_data
#print chi_data


In [55]:
def chi_cond(c_layer, var):
    """chi-square test for conditional independencies"""
    for cond in c_layer:
        cats = data_binned.get(cond).cat.categories
        print "categories: ", cats
        for c in range(len(cats)):
            print "bin: ", cats[c]
            print "conditional variable: ", cond
            data_binned_cond = data_binned.get(data_binned.get(cond).cat.codes == c)
            vals_month, counts_month = np.unique(data_binned_cond.get(var), return_counts=True)
            vals_area, counts_area = np.unique(data_binned_cond.get("area"), return_counts=True)

            table = pd.crosstab(data_binned_cond.get("area"),data_binned_cond.get(var))

            print table

            #try - except omdat sommige levels van rain niet te conditioneren zijn - geven voornamelijk nul-frequenties
            try:
                chisq, p, _, _ = stats.chi2_contingency(table,correction=True) #leuke error hierzo 
                print "chisquare: ", chisq, "\np-value: ", p
            except ValueError:
                print "error: zero frequenties"
                pass

In [56]:
#chi-square tests of conditional independence month - area 

l2 = ["wind","rain","RH","temp"]
#print data_binned.get("wind").cat.categories

chi_cond(l2,"month")
        
#temp+wind hebben p<0.05 voor sommige levels dus month is conditionally dependent of area given wind/temp. 
#rain+RH hebben p>0.05 voor alle levels dus month is conditionally independent of area given rain/RH; 
#de pijlen van rain+RH kunnen dus worden weggehaald

categories:  IntervalIndex([(0.391, 2.2], (2.2, 4.0], (4.0, 5.8], (5.8, 7.6], (7.6, 9.4]]
              closed='right',
              dtype='interval[float64]')
bin:  (0.391, 2.2]
conditional variable:  wind
month            apr  aug  feb  jan  jul  jun  mar  oct  sep
area                                                        
(-0.001, 0.52]     1   15    3    1    4    2    3    3   24
(0.52, 1090.84]    0   16    4    0    5    1    1    0   29
chisquare:  7.09125776503 
p-value:  0.526819465962
bin:  (2.2, 4.0]
conditional variable:  wind
month            apr  aug  feb  jan  jul  jun  mar  may  oct  sep
area                                                             
(-0.001, 0.52]     2   36    6    1    6    1    8    0    3   33
(0.52, 1090.84]    1   36    1    0    5    3    7    1    4   34
chisquare:  7.13824523276 
p-value:  0.622728586998
bin:  (4.0, 5.8]
conditional variable:  wind
month            apr  aug  dec  feb  jul  jun  mar  may  nov  oct  sep
area               

In [57]:
#chi-square tests of conditional independence rain - area
l3_1 = ["FFMC","DMC"]
#l3_2 = ["DMC","ISI"] #even kijken hoe dat zit met het feit dat FFMC de ISI beinvloedt
chi_cond(l3_1,"rain")

#resultaten: zeer sterke (p=1.0) conditional INdependence rain - area given FFMC/DMC

categories:  IntervalIndex([(18.699, 90.2], (90.2, 91.6], (91.6, 92.9], (92.9, 96.2]]
              closed='right',
              dtype='interval[float64]')
bin:  (18.699, 90.2]
conditional variable:  FFMC
rain             (-0.0064, 3.2]
area                           
(-0.001, 0.52]               70
(0.52, 1090.84]              62
chisquare:  0.0 
p-value:  1.0
bin:  (90.2, 91.6]
conditional variable:  FFMC
rain             (-0.0064, 3.2]
area                           
(-0.001, 0.52]               65
(0.52, 1090.84]              65
chisquare:  0.0 
p-value:  1.0
bin:  (91.6, 92.9]
conditional variable:  FFMC
rain             (-0.0064, 3.2]
area                           
(-0.001, 0.52]               69
(0.52, 1090.84]              66
chisquare:  0.0 
p-value:  1.0
bin:  (92.9, 96.2]
conditional variable:  FFMC
rain             (-0.0064, 3.2]  (3.2, 6.4]
area                                       
(-0.001, 0.52]               55           0
(0.52, 1090.84]              64           1


In [58]:
#chi-square tests of conditional independence temp - area
chi_cond(l3_1,"temp")
#RESULTS: temp conditionally INdependent of area given FFMC/DMC

categories:  IntervalIndex([(18.699, 90.2], (90.2, 91.6], (91.6, 92.9], (92.9, 96.2]]
              closed='right',
              dtype='interval[float64]')
bin:  (18.699, 90.2]
conditional variable:  FFMC
temp             (2.199, 15.5]  (15.5, 19.3]  (19.3, 22.8]  (22.8, 33.3]
area                                                                    
(-0.001, 0.52]              43            17             8             2
(0.52, 1090.84]             38            10             7             7
chisquare:  4.49958013313 
p-value:  0.212327741584
bin:  (90.2, 91.6]
conditional variable:  FFMC
temp             (2.199, 15.5]  (15.5, 19.3]  (19.3, 22.8]  (22.8, 33.3]
area                                                                    
(-0.001, 0.52]              13            28            17             7
(0.52, 1090.84]             19            16            22             8
chisquare:  5.10541958042 
p-value:  0.164238569052
bin:  (91.6, 92.9]
conditional variable:  FFMC
temp        

In [60]:
##chi-square tests of conditional independence RH - area
chi_cond(l3_1,"RH")
#RESULTS: RH conditionally independent of area given FFMC/DMC

categories:  IntervalIndex([(18.699, 90.2], (90.2, 91.6], (91.6, 92.9], (92.9, 96.2]]
              closed='right',
              dtype='interval[float64]')
bin:  (18.699, 90.2]
conditional variable:  FFMC
RH               (14.999, 33.0]  (33.0, 42.0]  (42.0, 53.0]  (53.0, 100.0]
area                                                                      
(-0.001, 0.52]                8            15            23             24
(0.52, 1090.84]              12            10            13             27
chisquare:  4.28513960423 
p-value:  0.232274832445
bin:  (90.2, 91.6]
conditional variable:  FFMC
RH               (14.999, 33.0]  (33.0, 42.0]  (42.0, 53.0]  (53.0, 100.0]
area                                                                      
(-0.001, 0.52]               15            13            15             22
(0.52, 1090.84]               9            23            16             17
chisquare:  4.95106148332 
p-value:  0.175415938146
bin:  (91.6, 92.9]
conditional variable:  F

In [61]:
l3_2 = ["DMC","ISI"] #even kijken hoe dat zit met het feit dat FFMC de ISI beinvloedt
chi_cond(l3_2,"wind")
#RESULTS: conditionally DEPENDENT of area given FFMC/ISI

categories:  IntervalIndex([(1.099, 68.6], (68.6, 108.3], (108.3, 142.4], (142.4, 291.3]]
              closed='right',
              dtype='interval[float64]')
bin:  (1.099, 68.6]
conditional variable:  DMC
wind             (0.391, 2.2]  (2.2, 4.0]  (4.0, 5.8]  (5.8, 7.6]  (7.6, 9.4]
area                                                                         
(-0.001, 0.52]             11          25          25          10           1
(0.52, 1090.84]             8          16          19           3          12
chisquare:  15.0107962622 
p-value:  0.00467887728165
bin:  (68.6, 108.3]
conditional variable:  DMC
wind             (0.391, 2.2]  (2.2, 4.0]  (4.0, 5.8]  (5.8, 7.6]  (7.6, 9.4]
area                                                                         
(-0.001, 0.52]             15          29          18           5           1
(0.52, 1090.84]            12          23          22           5           1
chisquare:  1.23660280351 
p-value:  0.872036219314
bin:  (108.3, 1

In [8]:
# fit model on data
G.fit(data_binned_train) # only used for discrete bayesnets?

KeyboardInterrupt: 

In [ ]:
G.get_cpds()

In [ ]:
G.get_independencies()

In [ ]:
predictions = G.predict(data_binned_test)
predictions.head()

In [ ]:
predictions = predictions["area"].astype('category').cat.codes

In [ ]:
test_labels = test_labels.astype('category').cat.codes

In [ ]:
y_pred = np.array(predictions)
y_true = np.array(test_labels)

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_true, y_pred)
acc

Hier komt de code die het als LGBN runt zodat het continue variabelen kunnen zijn. Is trial and error dus no flame pls.

To instantiate an object of this class, one needs to provide a variable name, the value of the beta_0 term, the variance, a list of the parent variable names and a list of the coefficient values of the linear equation (beta_vector), where the list of parent variable names and beta_vector list is optional and defaults to None.

In [ ]:
from pgmpy.models import LinearGaussianBayesianNetwork
from pgmpy.factors.continuous import LinearGaussianCPD

In [ ]:
# prepocessing data
#make dummy variables from X,Y,Month,Day
data_with_dummies = pd.get_dummies(data, columns=["X","Y"], prefix=["X","Y"])
data_with_dummies = pd.get_dummies(data_with_dummies, columns=["month","day"])
data_with_dummies.head()

In [ ]:
#calculations here
#calculate variance of each column
variances = np.var(data,0)
print(variances)
#variances.get('X')

In [ ]:
model = LinearGaussianBayesianNetwork()

#Instantiation of data points
#cpd = LinearGaussianCPD(name, beta_0 term, variance, list_parent_names, list_coeff_val_lin_eq(beta vector)) last 2 are optional

model.add_nodes_from(data_with_dummies)
model.add_edges_from([("temp","area"),
                  ("wind","area"),
                  ("rain","area"),
                  ("RH","area"),
                  ("FFMC","area"),
                  ("DMC","area"),
                  ("ISI","area"),
                  ("RH","FFMC"),
                  ("wind","FFMC"),
                  ("rain","FFMC"),
                  ("temp","FFMC"),
                  ("RH","DMC"),
                  ("rain","DMC"),
                  ("temp","DMC"),
                  ("FFMC","ISI"),
                  ("wind","ISI")])
model.add_edges_from([(col,"area") for col in data_with_dummies if col.startswith('X')])
model.add_edges_from([(col,"rain") for col in data_with_dummies if col.startswith('Y')])
model.add_edges_from([(col,"RH") for col in data_with_dummies if col.startswith('day')])
model.add_edges_from([(col,"temp") for col in data_with_dummies if col.startswith('month')])
model.add_edges_from([(col,"wind") for col in data_with_dummies if col.startswith('month')])

estimator = MaximumLikelihoodEstimator(model,data_with_dummies)

#IMPORTANT: month should be replaced with the dummy variables of month, like below
#likewise for X, Y, day in cpd_area
#beta_0 = estimator.estimate_cpd('temp')
cpd_temp = LinearGaussianCPD('temp', estimator.estimate_cpd('temp').get_values(), variances.get('temp'), ['month'])
cpd_wind = LinearGaussianCPD('wind', estimator.estimate_cpd('wind'), variances.get('wind'), ['month'])
cpd_rain = LinearGaussianCPD('rain', estimator.estimate_cpd('rain'), variances.get('rain'), ['month'])
cpd_RH = LinearGaussianCPD('RH', estimator.estimate_cpd('RH'), variances.get('RH'), ['month'])
cpd_FFMC = LinearGaussianCPD('FFMC', estimator.estimate_cpd('FFMC'), variances.get('FFMC'), ['rain','RH','temp','wind'])
cpd_DMC = LinearGaussianCPD('DMC', estimator.estimate_cpd('DMC'), variances.get('DMC'), ['rain','RH','temp'])
cpd_ISI = LinearGaussianCPD('ISI', estimator.estimate_cpd('ISI'), variances.get('ISI'), ['FFMC','wind'])
cpd_area = LinearGaussianCPD('area', estimator.estimate_cpd('area'), variances.get('area'), ['rain','RH','temp','wind','month','FFMC','DMC','ISI','day','X','Y'])
#should every dummy var have a data point? think so but not sure, 44 in total (incl above)

#jgd = model.to_joint_gaussian()

#model.fit(data_with_dummies)


In [ ]:
tmp = estimator.estimate_cpd('temp').get_values()

In [ ]:
print(tmp)
print np.shape(tmp)

In [ ]:
[('x1', 'x2'), ('x2', 'x3')]
cpd1 = LinearGaussianCPD('x1', 1, 4)
cpd2 = LinearGaussianCPD('x2', -5, 4, ['x1'], [0.5])
cpd3 = LinearGaussianCPD('x3', 4, 3, ['x2'], [-1])
model.add_cpds(cpd1, cpd2, cpd3)
jgd = model.to_joint_gaussian()
jgd.variables

#structure score for eval 